# (Required Imports)

In [ ]:
# TODO(deflaux): remove this cell after gcr.io/broad-ml4cvd/deeplearning:tf2-latest-gpu has this preinstalled.
from ml4h.runtime_data_defines import determine_runtime
from ml4h.runtime_data_defines import Runtime

if Runtime.ML4H_VM == determine_runtime():
  !pip3 install --user ipycanvas==0.7.0 ipyannotations==0.2.1
  !jupyter nbextension install --user --py ipycanvas
  !jupyter nbextension enable --user --py ipycanvas
  # Be sure to restart the kernel if pip installs anything.
  # Also, shift-reload the browser page after the notebook extension installation.

# Restart kernel here !!!

In [ ]:
from ipyannotations import BoxAnnotator, PointAnnotator, PolygonAnnotator
from ipyannotations.images.annotator import Annotator
from ml4h.visualization_tools.annotation_storage import DataFrameAnnotationStorage
from ml4h.visualization_tools.annotations import T1MAPPointAnnotator
from ml4h.visualization_tools.batch_image_annotations import BatchImageAnnotator
import pandas as pd
import tensorflow as tf

pd.set_option('display.max_colwidth', -1)

In [ ]:
%%javascript
// Display cell outputs to full height (no vertical scroll bar)
IPython.OutputArea.auto_scroll_threshold = 9999;

# Define the batch of samples to annotate

In [ ]:
USER = 'mklarqvist'

png_df = pd.read_csv(f'png_split_{USER}.csv')
png_df

### Use code below to eventually exclude images that you may have already annotated

In [ ]:
# List the dataframes already annotated by the user
import glob
annotated_by_user = glob.glob(f'goodbad_t1map_inference_{USER}*')
annotated_by_user

In [ ]:
# If you want to exclude any of the images from the CSVs above, please add the filenames (separatedy by commas) 
# to the 'to_exclude' list
to_exclude = [
    
]

In [ ]:
# Remove images in to_exclude
for to_exclude_csv in to_exclude:
    to_exclude_df = pd.read_csv(to_exclude_csv)
    png_df = pd.merge(png_df, to_exclude_df[['sample_id', 'instance_number']], indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1).reset_index(drop=True)
png_df

# Annotate the batch! 

In [ ]:
# Note: a zoom level of 1.0 displays the tensor as-is. For higher zoom levels, this code currently
# use the PIL library to scale the image.
import datetime

output_filename = f'goodbad_t1map_inference_{USER}_{datetime.datetime.now().strftime("%m_%d_%y_%H_%M_%S")}.csv'

annotator = BatchImageAnnotator(samples=png_df,
                                zoom=1.5,
                                annotation_categories=['good', 'bad', 'review'],
                                annotation_storage=DataFrameAnnotationStorage(output_filename),
                                annotator=T1MAPPointAnnotator,
                                user=USER
                               )
annotator.annotate_images()

# View the stored annotations 

In [ ]:
annotator.view_recent_submissions(count=100)

# Provenance

In [ ]:
import datetime
print(datetime.datetime.now())

In [ ]:
%%bash
pip3 freeze

In [ ]:
annotator.annotation_widget.data

Questions about these particular notebooks? Join the discussion https://github.com/broadinstitute/ml4h/discussions.